In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('0_raw/raw_data.csv')

In [14]:
df.sample(10)

,id,name,email,date_of_birth,signup_date,subscription_status
58983,58984,Danielle Clark,jreid@example.net,1981-08-25,2021-10-10,active
43414,43415,Jeffrey Espinoza,leetammyexample.net,NaN,2022-03-13,active
68911,68912,NaN,NaN,2006-02-27,2023-07-14,inactive
64695,64696,Kristina Brown,fclarkexample.org,NaN,2023-06-04,inactive
40163,40164,Kim Ho,smithmelissaexample.org,1943-12-31,2024-04-05,active
30801,30802,Danielle White,roger26example.org,NaN,2021-12-08,active
44579,44580,NaN,NaN,1936-02-26,2024-03-01,active
13322,13323,Michael Roth,yserranoexample.com,2000-07-18,2020-06-28,active
18172,18173,Natalie Davidson,ygray@example.org,1975-03-15,2024-05-31,inactive
60837,60838,Tammy Boone,william39example.net,1951-03-25,2023-12-27,active


In [ ]:
import re

def email_at_fill(email):
    """
    Fixes emails by adding an "@" before "example", "gmail", "hotmail", "outlook", "yahoo" if it's missing.
    """
    if isinstance(email, str):
        domains = ["example", "gmail", "hotmail", "outlook", "yahoo"]
        for domain in domains:
            if domain in email and "@" not in email:
                email = email.replace(domain, "@" + domain)
        # Basic email validation using regex
        if not re.match(r"[^@]+@[^@]+\.[^@]+", email):
            return None  # Or handle invalid emails as needed
        return email
    return None  # Handle non-string inputs

# Example usage:
# df['email'] = df['email'].apply(fix_email)

def email_dotcom_fill(email):
    """
    Appends '.com' to emails if they are missing it.
    """
    if isinstance(email, str):
        if ".com" not in email:
            # Basic check to avoid adding .com to already valid emails
            if re.match(r"[^@]+@[^@]+\.[^@]+", email):
                return email
            elif "@" in email:
                email = email.strip()
                email = email + ".com"
        return email
    return None

# Example usage:
# df['email'] = df['email'].apply(fill_missing_dotcom)

def merge_duplicate_emails(df):
    """
    Merges records with duplicate emails, filling missing data from duplicates, and removes duplicates.
    """
    # Find duplicate emails
    duplicate_emails = df[df['email'].duplicated(keep=False)]

    # Iterate through duplicate emails
    for email in duplicate_emails['email'].unique():
        # Get all rows with the same email
        email_group = df[df['email'] == email]

        if len(email_group) > 1:
            # Get the index of the first row (the one we'll merge into)
            first_index = email_group.index[0]

            # Iterate through the other rows and fill NaN values in the first row
            for index in email_group.index[1:]:
                for col in df.columns:
                    if pd.isna(df.loc[first_index, col]) and not pd.isna(df.loc[index, col]):
                        df.loc[first_index, col] = df.loc[index, col]

    # Remove duplicate rows based on email
    df = df.drop_duplicates(subset=['email'], keep='first')

    return df

df = merge_duplicate_emails(df)

In [ ]:
def email_validation(email):
    """
    Validates email addresses using regex.
    """
    if isinstance(email, str):
        # Basic email validation using regex
        if re.match(r"[^@]+@[^@]+\.[^@]+", email):
            return True
        else:
            return False
    return False  # Handle non-string inputs

In [10]:
df["subscription_status"].value_counts()

subscription_status
inactive    50081
active      49918
Name: count, dtype: int64

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99999 entries, 0 to 99998
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   99999 non-null  int64 
 1   name                 75113 non-null  object
 2   email                74953 non-null  object
 3   date_of_birth        74828 non-null  object
 4   signup_date          99998 non-null  object
 5   subscription_status  99999 non-null  object
dtypes: int64(1), object(5)
memory usage: 4.6+ MB


In [7]:
df['email'].duplicated().sum()

np.int64(30084)

raw -> Bronze
- Tratamento de duplicatas
- Tratamento de nulos
- Transformação de datas em datetime
- Subscription status as boolean?
- validação dos emails usando regex. Corrigir os erros mais comuns
    - Falta do @. Colocar antes da string 'example'
    - Falta do .com
- ordenação por signup date

Bronze -> Prata
- Criação da feature age
- Criação da feature signup_time 